In [ ]:
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
#| hide
#| default_exp medrxiv

# medRxiv

> The preprint server for health sciences.

In [ ]:
#| export

import warnings
warnings.filterwarnings("ignore")

import os

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
#| export

url = "https://connect.medrxiv.org/medrxiv_xml.php?subject=all"
response = requests.get(url)
response.raise_for_status()

In [ ]:
#| export

soup = BeautifulSoup(response.content, "lxml-xml")
items = soup.find_all("item")

In [ ]:
#| export

data = []
for item in items:
    item_data = {}
    for child in item.find_all(recursive=False):
        tag_name = child.name
        tag_value = child.text.strip() if child.text else None
        item_data[tag_name] = tag_value
    item_data.update(item.attrs)
    data.append(item_data)
data[0]

In [ ]:
#| export

filepath = "../data/medrxiv.json"
os.makedirs(os.path.dirname(filepath), exist_ok=True)

new_data = pd.DataFrame(data)
if os.path.exists(filepath):
    existing_data = pd.read_json(filepath, lines=True)
    combined_data = pd.concat([existing_data, new_data])
else:
    combined_data = new_data

combined_data.drop_duplicates(subset="identifier").to_json(filepath, orient="records", lines=True)
print(os.path.abspath(filepath))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()